## Ha Dinh's Note book

### Import lib and file

In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

import os
for dirname, _, filenames in os.walk('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/All_in_one_table.xlsx
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_customers_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_geolocation_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_orders_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_order_items_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_order_payments_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_order_reviews_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_products_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_sellers_dataset.csv
D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/product_category_name_translation.csv


In [13]:
 #load all the datasets:

customers_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_customers_dataset.csv', engine='python', encoding = "utf-8-sig")
geolocation_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_geolocation_dataset.csv', engine='python', encoding = "utf-8-sig")
orders_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_orders_dataset.csv', engine='python', encoding = "utf-8-sig")
order_items_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_order_items_dataset.csv', engine='python', encoding = "utf-8-sig")
order_payments_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_order_payments_dataset.csv', engine='python', encoding = "utf-8-sig")
order_reviews_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_order_reviews_dataset.csv', engine='python', encoding = "utf-8-sig")
products_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_products_dataset.csv', engine='python', encoding = "utf-8-sig")
sellers_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/olist_sellers_dataset.csv', engine='python', encoding = "utf-8-sig")
category_transalations_df = pd.read_csv('D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/product_category_name_translation.csv', engine='python', encoding = "utf-8-sig")

In [14]:
customers_df.head(5)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [15]:
# Check the size of each df:
df_names = ['customers_df','geolocation_df', 'orders_df', 'order_items_df','order_payments_df',
            'order_reviews_df','products_df','sellers_df','category_transalations_df']
for df in df_names:
    print("Dataset {} has shape {}".format(df, eval(df).shape))

Dataset customers_df has shape (99441, 5)
Dataset geolocation_df has shape (1000163, 5)
Dataset orders_df has shape (99441, 8)
Dataset order_items_df has shape (112650, 7)
Dataset order_payments_df has shape (103886, 5)
Dataset order_reviews_df has shape (100000, 7)
Dataset products_df has shape (32951, 9)
Dataset sellers_df has shape (3095, 4)
Dataset category_transalations_df has shape (71, 2)


In [18]:
for df in df_names:
    print("Number of null values from {} is {}".format(df, eval(df).isnull().sum().sum()))

Number of null values from customers_df is 0
Number of null values from geolocation_df is 0
Number of null values from orders_df is 4908
Number of null values from order_items_df is 0
Number of null values from order_payments_df is 0
Number of null values from order_reviews_df is 146532
Number of null values from products_df is 2448
Number of null values from sellers_df is 0
Number of null values from category_transalations_df is 0


*There are some null values from orders_df, order_reviews_df and products_df, we need to find the way to treat those values*

### Data Pre-processing

In [131]:
# import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table to the sqlite engine
customers_df.to_sql("customers", con =engine)
sellers_df.to_sql("sellers", con =engine)
order_reviews_df.to_sql("order_reviews", con =engine)
order_items_df.to_sql("order_items", con =engine)
products_df.to_sql("products", con =engine)
geolocation_df.to_sql("geolocation", con =engine)
category_transalations_df.to_sql("category_transalations", con =engine)
orders_df.to_sql("orders", con =engine)
order_payments_df.to_sql("order_payments", con =engine)

**1.Orders**


In [38]:
orders_df.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [106]:
sql ='''
select * 
from orders 
where order_approved_at is not null 
and order_delivered_carrier_date is not null
and order_delivered_customer_date is not null
limit 10; '''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
5,5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01 00:00:00
6,6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,2017-04-13 13:25:17,2017-05-09 00:00:00,2017-05-09 00:00:00
7,7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07 00:00:00
8,8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-01-25 02:50:47,2017-01-26 14:16:31,2017-02-02 14:08:10,2017-03-06 00:00:00
9,9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29 11:55:02,2017-07-29 12:05:32,2017-08-10 19:45:24,2017-08-16 17:14:30,2017-08-23 00:00:00


In [90]:
# Find out what is the relation between deliver_carier_date and order_approve_date
sql ='''
select count(*) as total,sum(case when Cast ((JulianDay(deliver_carier_date) - JulianDay(approve_time)) As Integer) = 0 then 1 else 0 end) as equal,
sum(case when Cast ((JulianDay(deliver_carier_date) - JulianDay(approve_time)) As Integer) > 0 then 1 else 0 end) as later,
sum(case when Cast ((JulianDay(deliver_carier_date) - JulianDay(approve_time)) As Integer) < 0 then 1 else 0 end) as sooner
from
(select strftime('%Y-%m-%d',order_delivered_carrier_date) as deliver_carier_date,strftime('%Y-%m-%d',order_approved_at) as approve_time
from orders 
where order_approved_at is not null 
and order_delivered_carrier_date is not null 
and order_delivered_customer_date is not null)

'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(5)

,total,equal,later,sooner
0,96461,15043,80744,674


In [89]:
# Find out what is the relation between order_delivered_customer_date and order_estimated_delivery_date
sql ='''
select count(*) as total,sum(case when Cast ((JulianDay(deliver_time) - JulianDay(estimated_time)) As Integer) = 0 then 1 else 0 end) as equal,
sum(case when Cast ((JulianDay(deliver_time) - JulianDay(estimated_time)) As Integer) > 0  then 1 else 0 end) as Later,
sum(case when Cast ((JulianDay(deliver_time) - JulianDay(estimated_time)) As Integer) < 0 then 1 else 0 end) as Sooner
from
(select strftime('%Y-%m-%d',order_delivered_customer_date) as deliver_time,strftime('%Y-%m-%d',order_estimated_delivery_date) as estimated_time
from orders 
where order_approved_at is not null 
and order_delivered_carrier_date is not null 
and order_delivered_customer_date is not null)

'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(5)

,total,equal,Later,Sooner
0,96461,1292,6534,88635


*1. Order approval follows the order purchase and usually happens a short time, hence missing approval timestamp can be replaced with the purchase timestamp*

*2. Nearly 99% of order_delivered_carrier_date is close to order_approved_at, so we can replace deliver timestamp by order_approved_at 

*3. More than 90% of order_delivered_customer_date is sooner or equal than order_estimated_delivery_date, so we can replace deliver timestamp by order_estimated_delivery_date 

In [91]:
orders_df["order_approved_at"] = orders_df["order_approved_at"].fillna(orders_df["order_purchase_timestamp"])
orders_df["order_delivered_carrier_date"] = orders_df["order_delivered_carrier_date"].fillna(orders_df["order_approved_at"])
orders_df["order_delivered_customer_date"] = orders_df["order_delivered_customer_date"].fillna(orders_df["order_estimated_delivery_date"])

In [92]:
#recheck
orders_df.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

**2.Orders reviews**

In [94]:
order_reviews_df.isnull().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       88285
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [97]:
sql ='''
select * 
from order_reviews
where review_comment_title is null
or review_comment_message is null
'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(5)

,index,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


*The null values in  review_comment_title and review_comment_message are where customer did not give a comment with their reviews => We can replace it with NONE*

In [100]:
order_reviews_df['review_comment_title'] = order_reviews_df['review_comment_title'].fillna('NONE')
order_reviews_df['review_comment_message'] = order_reviews_df['review_comment_message'].fillna('NONE')

In [101]:
#recheck
order_reviews_df.isnull().sum()

review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64

**3.Products**

In [104]:
products_df.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [105]:
sql ='''
select * 
from products
where product_category_name is null
or product_name_lenght      is null
or product_description_lenght  is null
or product_photos_qty  is null
'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(5)

,index,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,105,a41e356c76fab66334f36de622ecbd3a,None,None,None,None,650.0,17.0,14.0,12.0
1,128,d8dee61c2034d6d075997acef1870e9b,None,None,None,None,300.0,16.0,7.0,20.0
2,145,56139431d72cd51f19eb9f7dae4d1617,None,None,None,None,200.0,20.0,20.0,20.0
3,154,46b48281eb6d663ced748f324108c733,None,None,None,None,18500.0,41.0,30.0,41.0
4,197,5fb61f482620cb672f5e586bb132eae9,None,None,None,None,300.0,35.0,7.0,12.0


*For the product catalog, we can replace category's name or description or name by other products's information base on their dimention => Drop those products*

In [112]:
sql ='''
select sum(payment_value)
from
(select order_id, payment_value from order_payments)a
inner join
(select order_id, product_id from order_items)b
on a.order_id = b.order_id
inner join 
(select product_id from products
where product_category_name is null
or product_name_lenght      is null
or product_description_lenght  is null
or product_photos_qty  is null) c
on b.product_id = c.product_id
'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(5)

,sum(payment_value)
0,252801.71


In [114]:
sql ='''
select sum(payment_value) 
 from order_payments

'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(5)

,sum(payment_value)
0,1.600887e+07


*The total charge of null product only accounts for ~1.5% => We can drop those products for a clear analysis*

In [117]:
products_df = products_df[~(products_df['product_name_lenght'].isnull())]

In [119]:
#recheck
products_df.isnull().sum()

product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              1
product_length_cm             1
product_height_cm             1
product_width_cm              1
dtype: int64

In [121]:
products_df[products_df['product_height_cm'].isnull()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,60.0,865.0,3.0,NaN,NaN,NaN,NaN


*=> Only 1 product without dimension can be keep for further analysis*

### Data Processing

In [122]:
geolocation_df.head(5)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [127]:
# Merge all the datasets into 1 

df = pd.merge(orders_df,order_payments_df, on="order_id")
df = pd.merge(df,customers_df, on="customer_id")
df = pd.merge(df,order_items_df, on="order_id")
df = pd.merge(df,sellers_df, on="seller_id")
df = pd.merge(df,order_reviews_df, on="order_id")
df = pd.merge(df,products_df, on="product_id")
df = pd.merge(df,category_transalations_df, on="product_category_name")

df.to_sql("all_in_one_table", con =engine)
df.shape

(116581, 40)

In [134]:
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,review_answer_timestamp,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,...,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3,voucher,...,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2,voucher,...,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1,credit_card,...,2017-08-20 15:16:36,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1,credit_card,...,2017-08-08 23:26:23,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares


#### To maximize GMV, we can do some analysis:
- Optimize products base on purchasing and customer's review
- Optimize transportation's goods
- Find out the purchasing behavior from customers
- Find the best seller, state to focus on them in the future

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116581 entries, 0 to 116580
Data columns (total 40 columns):
order_id                         116581 non-null object
customer_id                      116581 non-null object
order_status                     116581 non-null object
order_purchase_timestamp         116581 non-null object
order_approved_at                116581 non-null object
order_delivered_carrier_date     116581 non-null object
order_delivered_customer_date    116581 non-null object
order_estimated_delivery_date    116581 non-null object
payment_sequential               116581 non-null int64
payment_type                     116581 non-null object
payment_installments             116581 non-null int64
payment_value                    116581 non-null float64
customer_unique_id               116581 non-null object
customer_zip_code_prefix         116581 non-null int64
customer_city                    116581 non-null object
customer_state                   116581 non-null ob

In [145]:
sql ='''
select * 
from order_payments
where payment_installments > 1

'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(5)

,index,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
2,4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
3,5,298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.12
4,7,3d7239c394a212faae122962df514ac7,1,credit_card,3,51.84


In [151]:
sql ='''
select count(distinct order_id) as number_of_buy,
sum(payment_value *payment_installments) as total_charge,
seller_state
from all_in_one_table
group by seller_state
order by total_charge desc
'''
sql_exc = pd.read_sql_query(sql,con=engine)
sql_exc.head(30)


,number_of_buy,total_charge,seller_state
0,69151,5.378033e+07,SP
1,7621,7.543856e+06,PR
2,7759,5.973026e+06,MG
3,4273,4.881520e+06,RJ
4,3650,4.022176e+06,SC
5,1949,2.418852e+06,RS
6,569,2.225627e+06,BA
7,815,5.531865e+05,DF
8,406,5.107896e+05,PE
9,453,3.509513e+05,GO


In [154]:
df.to_excel("D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/All_in_one_table.xlsx", engine='xlsxwriter')
df.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,review_answer_timestamp,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,...,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3,voucher,...,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2,voucher,...,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1,credit_card,...,2017-08-20 15:16:36,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1,credit_card,...,2017-08-08 23:26:23,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares


### Calculate RFM

In [164]:
# create frequency dataframe
frequency = df.groupby('customer_unique_id')['order_item_id'].count()
frequency = frequency.reset_index()
frequency.columns = ['customer_unique_id', 'frequency']

# create monetary dataframe
df['total_payment'] = df['payment_value'] * df['payment_installments']
monetary = df.groupby('customer_unique_id')['total_payment'].sum()
monetary = monetary.reset_index()
monetary.columns = ['customer_unique_id', 'monetary']

# create recency dataframe
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], infer_datetime_format=True, errors='ignore')
max_date = max(df['order_purchase_timestamp'])
df['diff_days'] =  (max_date-df['order_purchase_timestamp']).dt.days
recency = df.groupby('customer_unique_id')['diff_days'].min()
recency = recency.reset_index()
recency.columns = ['customer_unique_id', 'recency']

rfm_df = pd.merge(frequency, monetary, on='customer_unique_id', how='inner')
rfm_df = pd.merge(rfm_df, recency, on='customer_unique_id', how='inner')
rfm_df.head()

# removing (statistical) outliers for monetary
Q1 = rfm_df.monetary.quantile(0.05)
Q3 = rfm_df.monetary.quantile(0.95)
IQR = Q3 - Q1
rfm_df = rfm_df[(rfm_df.monetary >= Q1 - 1.5*IQR) & (rfm_df.monetary <= Q3 + 1.5*IQR)]

# outlier treatment for frequency
Q1 = rfm_df.frequency.quantile(0.05)
Q3 = rfm_df.frequency.quantile(0.95)
IQR = Q3 - Q1
rfm_df = rfm_df[(rfm_df.frequency >= Q1 - 1.5*IQR) & (rfm_df.frequency <= Q3 + 1.5*IQR)]

# outlier treatment for recency
Q1 = rfm_df.recency.quantile(0.05)
Q3 = rfm_df.recency.quantile(0.95)
IQR = Q3 - Q1
rfm_df = rfm_df[(rfm_df.recency >= Q1 - 1.5*IQR) & (rfm_df.recency <= Q3 + 1.5*IQR)]

#Normalized each of index
rfm_df['frequency_normalized'] = (rfm_df.frequency - rfm_df.frequency.mean())/rfm_df.frequency.std(ddof=0)
rfm_df['monetary_normalized'] = (rfm_df.monetary - rfm_df.monetary.mean())/rfm_df.monetary.std(ddof=0)
rfm_df['recency_normalized'] = (rfm_df.recency - rfm_df.recency.mean())/rfm_df.recency.std(ddof=0)

rfm_df.head(5)

,customer_unique_id,frequency,monetary,recency,frequency_normalized,monetary_normalized,recency_normalized
0,0000366f3b9a7992bf8c76cfdf3221e2,1,1135.20,115,-0.367949,0.494482,-0.828143
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,118,-0.367949,-0.531218,-0.808539
2,0000f46a3911fa3c0805444483337064,1,689.76,541,-0.367949,0.082132,1.955614
3,0000f6ccb0745a6a4b88665a16c9f078,1,174.48,325,-0.367949,-0.394870,0.544132
4,0004aac84e0df4da2b147fca70cf8255,1,1181.34,292,-0.367949,0.537195,0.328489


In [165]:
rfm_df.to_excel("D:/hadh12/Chuyên môn/Assignment/brazilian-ecommerce/RFM.xlsx", engine='xlsxwriter')
rfm_df.head(5)

,customer_unique_id,frequency,monetary,recency,frequency_normalized,monetary_normalized,recency_normalized
0,0000366f3b9a7992bf8c76cfdf3221e2,1,1135.20,115,-0.367949,0.494482,-0.828143
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,118,-0.367949,-0.531218,-0.808539
2,0000f46a3911fa3c0805444483337064,1,689.76,541,-0.367949,0.082132,1.955614
3,0000f6ccb0745a6a4b88665a16c9f078,1,174.48,325,-0.367949,-0.394870,0.544132
4,0004aac84e0df4da2b147fca70cf8255,1,1181.34,292,-0.367949,0.537195,0.328489
